In [1]:
import pandas as pd
def read_file(path):
    df= pd.read_csv(path).drop(columns=['city'])
    return df
read_file('message-weather (1).csv')

,date_weather,latitude_decimal,longitude_decimal,max_temperature,min_temperature,region_x,temperature,wind_direction,wind_speed,weather_description,date_hour_x,tg_message
0,2022-12-02 12:32,50.911944,34.803333,-0.2,-3.3,Сумська,-0.2,343,31.2,Overcast,2022-12-02 12:00:00,💥 Хотінь (Сумська обл.)\nЗагроза артилерійсько...
1,2022-12-02 11:33,47.644722,34.604167,2.7,-0.1,Дніпропетровська,2.7,3,28.5,Overcast,2022-12-02 11:00:00,💥 Марганець (Дніпропетровська обл.)\nЗагроза а...
2,2022-12-02 11:33,48.466111,35.025278,1.7,-1.6,Дніпропетровська,1.7,357,29.9,Partly cloudy,2022-12-02 11:00:00,💥 Марганець (Дніпропетровська обл.)\nЗагроза а...
3,2022-12-02 11:33,47.577222,34.357500,2.4,0.1,Дніпропетровська,2.4,7,25.4,Overcast,2022-12-02 11:00:00,💥 Марганець (Дніпропетровська обл.)\nЗагроза а...
4,2022-12-02 11:33,48.466111,35.025278,1.7,-1.6,Дніпропетровська,1.7,357,29.9,Partly cloudy,2022-12-02 11:00:00,💥 Марганець (Дніпропетровська обл.)\nЗагроза а...
...,...,...,...,...,...,...,...,...,...,...,...,...
8798,2023-04-01 09:06,46.640000,32.614444,18.1,10.2,Херсонська,16.9,248,45.4,Mainly clear,2023-04-01 09:00:00,Херсонська область. Інформація щодо ворожих об...
8799,2023-03-30 16:20,46.640000,32.614444,16.2,3.1,Херсонська,10.7,245,27.7,Mainly clear,2023-03-30 16:00:00,​​⚡️Отримав осколкове поранення під час бомбар...
8800,2023-03-10 08:43,46.640000,32.614444,-1.5,-6.3,Херсонська,-2.7,65,36.1,Overcast,2023-03-10 08:00:00,Херсонська область. Інформація щодо ворожих об...
8801,2023-03-10 08:43,46.640000,32.614444,-1.5,-6.3,Херсонська,-2.7,65,36.1,Overcast,2023-03-10 08:00:00,Херсонська область. Інформація щодо ворожих об...


In [47]:

def calculate_max_attacks(df):
    alarm_keywords = ['💥', '📍', '⚠️']
    df['alarm'] = df['tg_message'].apply(lambda x: any(keyword in x for keyword in alarm_keywords))
    df[['date', 'time']] = df['date_weather'].str.split(' ', n=1, expand=True)
    df['month'] = pd.to_datetime(df['date']).dt.month_name()
    df_count = df.groupby(['region_x', 'month']).size().reset_index(name='number_of_attacks')
    df_max_attacks = df_count.loc[df_count.groupby('region_x')['number_of_attacks'].idxmax()]
    df_max_attacks['region_x'] = df_max_attacks['region_x']
    df_max_attacks = df_max_attacks.sort_values(by=['region_x'])
    df_max_attacks = df_max_attacks.reset_index(drop=True)
    return df_max_attacks[['region_x', 'month', 'number_of_attacks']]
calculate_max_attacks(read_file('message-weather (1).csv'))

,region_x,month,number_of_attacks
0,Івано-Франківська,December,1
1,Вінницька,March,10
2,Дніпропетровська,November,636
3,Донецька,March,811
4,Житомирська,March,23
5,Запорізька,January,55
6,Київська,January,111
7,Кіровоградська,October,5
8,Луганська,March,251
9,Львівська,February,149


In [68]:
def process_weather_data(df):
    new_data = {'region_x': [], 'weather_description': [], 'tg_message_count': []}
    for _, row in df.iterrows():
        region = row['region_x']
        weather_description = row['weather_description']
        tg_messages = row['tg_message']

        for _ in tg_messages:
            new_data['region_x'].append(region)
            new_data['weather_description'].append(weather_description)
            new_data['tg_message_count'].append(1)
    result_df = pd.DataFrame(new_data)
    result_df = result_df.groupby(['region_x', 'weather_description']).agg({'tg_message_count':'sum'})

    return result_df
process_weather_data(read_file('message-weather (1).csv'))

tg_message_count
region_x          weather_description                  
Івано-Франківська Mainly clear                      179
                  Snow fall: Slight                 685
Вінницька         Clear sky                      100608
                  Drizzle: Light                    652
                  Mainly clear                     1247
...                                                 ...
Чернігівська      Drizzle: Light                   7932
                  Mainly clear                      714
                  Overcast                        10866
                  Partly cloudy                     861
                  Snow fall: Slight                4137

[131 rows x 1 columns]